In [1]:
import pandas as pd
import jira
import json

In [2]:
from input.jira_auth import username
from input.jira_auth import password

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
jira_client = jira.JIRA('https://creditsaison-in.atlassian.net',
    basic_auth=(username,password))

{'response': None, 'request': <PreparedRequest [GET]>}
{'response': None, 'request': <PreparedRequest [GET]>}
{'response': None, 'request': <PreparedRequest [GET]>}
{'response': None, 'request': <PreparedRequest [GET]>}


In [5]:
all_fields = jira_client.fields()

AttributeError: module 'jira' has no attribute 'fields'

In [38]:
jql = 'project in (CLS, VAULTFILE, DSR, DS, CICD, CW, IRMS, ITS, O2P, PD, PMM, TEC, VOCR) AND status in (Analyse, "Approved for Deployment", "Awaiting Approval", Backlog, Blocked, Brainstorming, "Data Acquisition", "Data Exploration", "Data Preparation", "Data Requirements", "Data Sources Identified", Development, Discover, Dormant, Draft, Enhancement, Evaluation, "Final Review", Ideation, "In Approval Stage", "In Architecture", "In Code Development", "In Development", "In Implementation", "In Peer Review", "In Progress", "In QA", "In RCA", "In Requirement", "In Research", "In Review", "In UAT", "Initial Research", "Manual Actions", "Not a Bug", "Not started", "On Hold", Open, "Peer Review", Performance, Plan, Planning, "Prep UAT", Prioritise, Prioritised, "Ready For Development", "Ready for Implementation", "Ready for Production", "Ready for QA", "Ready for review", "Ready to Release", "Regression / Sanity Testing", Research, "Result Analysis", Review, "Root Cause Analysis", "Selected for Development", "Tech In Progress", Testing, "Testing Phase", "To Do", "Under Review", "Waiting For Approval") AND assignee in (5c2f2bda9760f569b627a265, 616d477b2097220071947017, membersOf("Vaultron File Upload Group"), 622ac8bb9c3cce00694fde0e, 626a2f98593e80006f168ce2, 615ef3ec9cdb93007299c657, 601bf0e53b1af00069eb74ad, 5d91971975c86a0dcda2ecb3, 5f43765fea5e2f0039a6b79f) ORDER BY created DESC'


In [39]:
vfcr_jql = 'project = VFCR AND status in ("In Progress", "On Hold", Open, "Testing Phase", "To Do") order by created DESC'


In [14]:
# def execute_jql():
#     jira_issues = jira.search_issues(jql,maxResults=0)
#     issues = pd.DataFrame()
#     for issue in jira_issues:
#         d = {
#             'id':    issue.id,
#             'key':   issue.key,
#             'self':  issue.self,
        
#             'assignee':        str(issue.fields.assignee),
#             'creator' :        str(issue.fields.creator),
#             'reporter':        str(issue.fields.reporter),
#             'created' :        str(issue.fields.created),   
#             'labels':          str(issue.fields.labels),
#             'description':     str(issue.fields.description),
#             'summary':         str(issue.fields.summary),
#             'issuetype':       str(issue.fields.issuetype.name),
#             'priority':        str(issue.fields.priority.name),
#             'project':         str(issue.fields.project),
#             'resolution':      str(issue.fields.resolution),
#             'resolution_date': str(issue.fields.resolutiondate),
#             'status':          str(issue.fields.status.name),
#             'subtask':            str(issue.fields.issuetype.subtask),
#             'status_description': str(issue.fields.status.description)
#         }
#         issues = issues.append(d, ignore_index=True)
#     print(issues)
    

In [15]:
# execute_jql()

In [23]:
def fetch_issues(jql):
    block_size = 100
    block_num = 0
    allissues = []
    while True:
        start_idx = block_num*block_size
        issues = jira.search_issues(jql, start_idx, block_size)
        if len(issues) == 0:
         # Retrieve issues until there are no more to come
            break
        block_num += 1
        for issue in issues:
            #log.info('%s: %s' % (issue.key, issue.fields.summary))
            allissues.append(issue)  
    return allissues

In [24]:
cls_issues = fetch_issues(jql)
print("Size of cls issues:", len(cls_issues))

Size of cls issues: 184


In [25]:
vfcr_issues = fetch_issues(vfcr_jql)
print("Size of vfcr issues:", len(vfcr_issues))

Size of vfcr issues: 125


In [26]:
allissues = []
allissues.extend(cls_issues)
allissues.extend(vfcr_issues)

In [27]:
len(allissues)

309

In [29]:
jira_cols = [
     'project',
     'key', 
     'status',
     'labels',
     'issuelinks',
     'assignee',
     'creator',
     'reporter',
     'created',
     'description',
     'summary',
     'subtask',
     'issuetype',
     'priority',
     'statusDescription'
]

In [30]:
# jira_cols = list(jira_keys.keys())

In [31]:
extra_keys = {
    'productReady?': False,
     'productReadyDate': '',
     'implementationReady?': False,
     'implementationReadyDate': '',
     'implementationStartedDate': '',
     'implementationCompletionDate': '',
     'qaHandoverDate': '',
     'qaCompletionDate': '',
     'uatHandoverDate': '',
     'uatCompletionDate': '',
     'releaseDate': '',
     'plannedCompletionDate': '',
     'type[feature/bug/enhancement/refactor]': '',
     'FR/NFR': '',
     'blocker': '',
     'dependencyOn': '',
     'comments': '' 
}

In [33]:
## Into pandas
import pandas as pd
extra_cols = list(extra_keys.keys())
all_cols = []
all_cols.extend(jira_cols)
all_cols.extend(extra_cols)
new_issues = pd.DataFrame(columns=jira_cols)
for issue in allissues:
     d = {
     'project': issue.fields.project.name,
     'key': issue.key,
     'status': issue.fields.status.name,
     'labels': issue.fields.labels,
     'issuelinks': issue.fields.issuelinks,
     'assignee': issue.fields.assignee,
     'creator' : issue.fields.creator,
     'reporter': issue.fields.reporter,
     'created' : issue.fields.created,
     'description': issue.fields.description,
     'summary': issue.fields.summary,
     'subtask': issue.fields.issuetype.subtask,
     'issuetype': issue.fields.issuetype.name,
     'priority': issue.fields.priority.name,
     'statusDescription': issue.fields.status.description    
     }
     d_df = pd.DataFrame([d])
     d_df['taskurl'] = '=HYPERLINK("https://creditsaison-in.atlassian.net/browse/' + d_df["key"] +'","' + d_df["key"]+'")'
     new_issues = pd.concat([new_issues, d_df], ignore_index=True)
extra_cols.append('key')
try:
    issues=pd.read_excel('jira_task_list.xlsx')
    print("Existing issues count:", issues.shape[0])
    new_count = new_issues[~new_issues['key'].isin(issues['key'])].shape[0]
    print("New issues count:", new_count)
    issues = new_issues.merge(issues[extra_cols], on='key', how='left')
except FileNotFoundError as e:
    issues = new_issues.copy()
    issues = issues.merge(pd.DataFrame(columns=extra_cols), on='key', how='left')
issues.loc[issues['assignee'].isna(), 'assignee'] = 'Deepashree Kedia'
issues.to_excel('output/jira_task_list.xlsx', index=False, encoding='utf-8')
print("Total issues count:", issues.shape[0])
issues.head()

Total issues count: 309


,project,key,status,labels,issuelinks,assignee,creator,reporter,created,description,summary,subtask,issuetype,priority,statusDescription,taskurl,productReady?,productReadyDate,implementationReady?,implementationReadyDate,implementationStartedDate,implementationCompletionDate,qaHandoverDate,qaCompletionDate,uatHandoverDate,uatCompletionDate,releaseDate,plannedCompletionDate,type[feature/bug/enhancement/refactor],FR/NFR,blocker,dependencyOn,comments
0,Core Lending System,CLS-23160,In UAT,[],[],Deepashree Kedia,Chirag Garg,Chirag Garg,2022-04-29T11:35:42.009+0530,Accounting for MONTHLY INTEREST ACCRUAL,Accounting for MONTHLY INTEREST ACCRUAL,False,Vault,None,Owner: PM or QA or EM\nState: After story has ...,"=HYPERLINK(""https://creditsaison-in.atlassian....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Data Science & Risk : CS India,DSR-168,In Progress,[],[],Neelotpal Singh Gour,Neelotpal Singh Gour,Neelotpal Singh Gour,2022-04-28T13:33:37.773+0530,None,Make changes in code to write cross account in...,True,Data Sub task,High,This issue is being actively worked on at the ...,"=HYPERLINK(""https://creditsaison-in.atlassian....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Data Science & Risk : CS India,DSR-167,In Progress,[],[],Neelotpal Singh Gour,Neelotpal Singh Gour,Neelotpal Singh Gour,2022-04-28T13:32:00.996+0530,"As a part of this task, Create CFN template fo...",CFN template for E-agg TheMachine production d...,True,Data Sub task,High,This issue is being actively worked on at the ...,"=HYPERLINK(""https://creditsaison-in.atlassian....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Core Lending System,CLS-23139,Open,[],[],Abhisek Roy,Konica Garg,Konica Garg,2022-04-28T12:45:05.316+0530,Oracle Integration,Oracle Integration,False,Epic,High,The issue is open and ready for the assignee t...,"=HYPERLINK(""https://creditsaison-in.atlassian....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Data Science & Risk : CS India,DSR-161,On Hold,[TheMachine],[],Neelotpal Singh Gour,Neelotpal Singh Gour,Neelotpal Singh Gour,2022-04-25T12:01:59.707+0530,,Karza Employment Flattener Change,True,Data Sub task,Highest,Owner: EM or Dev or QA\r\nState: At any given ...,"=HYPERLINK(""https://creditsaison-in.atlassian....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
# for field_name in issue0.raw['fields']:
#     if field_name != 'parent':
#         print("Field:", field_name, "Value:", issue.raw['fields'][field_name])